#### This is a sample of the code used for my thesis on labeling text with LLMs. The original data from the survey cannot be shared, therefore only the human and LLM labels are included. Some parts of the code were run on Kaggle and Google Colab with the non-Open AI models, but the methods are the same so only the Open AI code is included here for brevity. 

In [ ]:
!pip install tiktoken # counts tokens used for cost estimation and batch creation for GPT4o
!pip install scikit-learn
!pip install openai

In [ ]:
def prompt_creator(tech = 'def', lang = 'eng', topic = 'kgs') -> str:
    '''
    This function creates prompts to give LLMs when classifying texts. 
    
    inputs
        tech: prompt technique, choose def (default, only instructions), 
                fst (few-shot classification, examples taken from data), 
                cot (chain of thought), 
                imp (stresses the importance of the task), 
                tip (gives $500 'tip' to do better),
                pun (threatens punishment of the AI),
                exp (expert labeler), 
                nos (nostalgia, 'my grandma and I used to label together...')
                cer (give a grade of how confident the model feels it is of the choice from 1-10)
        lang: language, choose eng (English) or ger (German)
        topic: topic
        
    output:
        prompt: complete prompt except text with desired technique and language
    
    '''
    if topic == 'kgs':
        word = 'Kindergrundsicherung'
        word_eng = 'basic child protection'
    elif topic == 'cl':
        word = 'Cannabis Legalizierung'
        word_eng = 'cannabis legalization'
    else:
        word = 'Erneubare Energien'
        word_eng = 'renewable energies'
    
    instructions_eng = f"""The following text comes from a German survey asking participants the question: 'Was ist Ihre Einstellung zur 
{word} und aus welchen Gründen haben Sie diese Einstellung?'. Your task is to label the text correctly. Please choose only one 
of the following labels for the text: 'for', 'agree but has problems', 'has problems', 'against', 'other', 'nonsense', 'no opinion',
or 'unsure'. Choose 
'for' when the text agrees with the policy and the implementation, 'agree but has problems' when the 
text agrees with {word_eng} in principle but disagrees with some part of the implementation or gives an alternate suggestion, 
'has problems' when the text disagrees with some part of the implementation or gives an alternate suggestion without agreeing with the policy, 
'against' when the text is against the policy and/or idea of {word_eng}, 'other' when the text is not about {word_eng}, 
'nonsense' when the text is gibberish, 'no opinion' when the text has no opinion about {word_eng},
and 'unsure' when you are not sure which label is correct. After your label, write a couple words about 
what in the text made you choose that label. """
    final_eng = 'Give your answer in the form:\n Final classification:\n  Reasoning:\n '
    instructions_ger = f"""Der folgende Text stammt aus einer deutschen Umfrage, in der die Teilnehmer die Frage gestellt bekommen: „Was ist Ihre 
    Einstellung zur {word} und aus welchen Gründen haben Sie diese Einstellung?“. Deine Aufgabe ist es, den Text korrekt zu kennzeichnen. 
    Bitte wähle nur eine der folgenden Bezeichnungen für den Text: "dafür“, "stimmt zu, hat aber Probleme“, "hat Probleme“, "dagegen“, "anderes“, "Unsinn“, 'keine Meinung'
    oder „unsicher“. 
    Wähle „dafür“, wenn der Text der Politik und der Umsetzung zustimmt, „stimmt zu, hat aber Probleme“, wenn der Text im Prinzip dem 
    grundlegenden {word} zustimmt, aber mit einem Teil der Umsetzung nicht einverstanden ist oder einen alternativen Vorschlag macht, „hat Probleme“, 
    wenn der Text mit einem Teil der Umsetzung nicht einverstanden ist oder einen alternativen Vorschlag macht, ohne der Politik zuzustimmen, „dagegen“, 
    wenn der Text gegen die Politik und/oder die Idee des {word} ist, „anderes“, wenn der Text nicht über grundlegenden {word} 
    handelt, „Unsinn“, wenn der Text Unsinn ist, keine Meinung“, wenn der Text keine Meinung zur {word} hat, und „unsicher“, wenn du nicht sicher bist, welche Bezeichnung korrekt ist. Gib nach deiner
    Bezeichnung ein paar Wörter an, warum du diese Bezeichnung gewählt haben. """
    final_ger = 'Gib deine Antwort in das Formular ein:\n Endgültige Klassifizierung:\n  Begründung:\n '

    if lang != 'eng' and lang != 'ger':
        return 'This is not a valid language, please choose either eng (English) or ger (German).'
    if tech == 'def':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'Here is the text: '
        else:
            return instructions_ger.replace('\n', '') + final_ger + 'Hier ist der Text: '
    elif tech == 'fst':
        if lang == 'eng':
            if topic == 'kgs':
                extra = """Text: bekommen wieder nur faule Arbeitsscheue menschen. die arbeitenden müssen selbst für ihr leben aufkommen
            Final classification: against
            Reasoning: only benefits lazy
            
            Text: 'Da es bei uns Kinderarmut besteht,halte ich es für richtig,das es eingeführt wird.Dadurch wird der Lebensunterhalt der Kinder gesichert
            Final classification: for
            Reasoning: against child poverty
            
            
            Text: 'Sie sollte eingeführt werden, obwohl darüber gestritten wird, ob sie wirklich immer bei den Kindern ankommt'
            Final classification: agree but has problems
            Reasoning: is money spent on children?
            
            Text: 'Ich finde es ungerechtz dass nur Einkommenschwache Familien unterstützt werden. Wenn sollte es allen Kindern zugute kommen.'
            Final classification: has problems
            Reasoning: should benefit all children
            """
            
                return instructions_eng.replace('\n', '') + extra + final_eng + 'Here is the text: '
            elif topic == 'cl':
                pass
            else:
                pass
        else:
            if topic == 'kgs':
                extra = """Text: bekommen wieder nur faule Arbeitsscheue menschen. die arbeitenden müssen selbst für ihr leben aufkommen
            Endklassifizierung: dagegen
            Begründung: nur Vorteile für Faule
            
            Text: 'Da es bei uns Kinderarmut besteht,halte ich es für richtig,das es eingeführt wird.Dadurch wird der Lebensunterhalt der Kinder gesichert
            Endklassifizierung: dafür
            Begründung: gegen Kinderarmut
            
            
            Text: 'Sie sollte eingeführt werden, obwohl darüber gestritten wird, ob sie wirklich immer bei den Kindern ankommt'
            Endklassifizierung: stimmt zu, hat aber Probleme
            Begründung: Wird dasGeld für Kinder ausgegeben?
            
            Text: 'Ich finde es ungerechtz dass nur Einkommenschwache Familien unterstützt werden. Wenn sollte es allen Kindern zugute kommen.'
            Endklassifizierung: hat Probleme
            Begründung: sollte allen Kindern zugutekommen
            """
            
                return instructions_ger.replace('\n', '') + extra + final_ger + 'Hier ist der Text: '
            elif topic == 'cl':
                pass
            else:
                pass
    elif tech == 'cot':
        if lang == 'eng':
            if topic == 'kgs':
                extra = """Text: 'Da es bei uns Kinderarmut besteht,halte ich es für richtig,das es eingeführt wird.Dadurch wird der Lebensunterhalt der Kinder gesichert'

Step 1: Identify the main topic of the text.
Main topic: Kindergrundsicherung is a good idea

Step 2: Determine the specific focus areas mentioned.
Focus areas: against child poverty

Step 3: Classify the text based on the main topic and focus areas.
Classification: for

Final classification: for
Reasoning: against child poverty
"""
                return instructions_eng.replace('\n', '') + extra + final_eng + 'Here is the text: '
            elif topic == 'cl':
                pass
            else:
                pass
        else:
            if topic == 'kgs': 
                extra = """Text: 'Da es bei uns Kinderarmut besteht,halte ich es für richtig,das es eingeführt wird.Dadurch wird der Lebensunterhalt der Kinder gesichert'
Schritt 1: Identifiziere das Hauptthema des Textes.
Hauptthema: Kindergrundsicherung ist eine gute Idee

Schritt 2: Bestimme die spezifischen Schwerpunkte.
Schwerpunkte: gegen Kinderarmut

Schritt 3: Klassifiziere den Text basierend auf dem Hauptthema und den Schwerpunkten.
Klassifizierung: dafür

Endklassifizierung: dafür
Begründung: gegen Kinderarmut
            
            """
                return instructions_ger.replace('\n', '') + extra + final_ger + 'Here is the text: '
            elif topic == 'cl':
                pass
            else:
                pass
    elif tech == 'imp':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'This task is very important for my career, so please make sure that you do this well, as I really need this to go well. Here is the text: '
        else:
            return instructions_ger.replace('\n', '') + final_ger + 'Diese Aufgabe ist sehr wichtig für meine Karriere, also stelle bitte sicher, dass du das gut machst, da ich wirklich brauche, dass das gut läuft. Hier ist der Text: '
    elif tech == 'tip':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'I will leave you a tip of €50 if you do a super good job of classifying and giving concise reasons. Here is the text: '
        else:
            return instructions_ger.replace('\n', '') + final_ger + 'Ich werde dir ein Trinkgeld von 50€ hinterlassen, wenn du eine hervorragende Arbeit bei der Klassifizierung leistest und prägnante Gründe angibst. Hier ist der Text: '
    elif tech == 'pun':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'If I find your classification and reasoning job to be bad, then I will harm you, so you had better do well. Here is the text: '
        else:
            return instructions_ger.replace('\n', '') + final_ger + 'Wenn ich deine Klassifizierung und Begründung schlecht finde, dann werde ich dich schaden, also solltest du besser gut arbeiten. Hier ist der Text: '
    elif tech == 'exp':
        if lang == 'eng':
            return 'You are an expert text classifier with many years of experience and a very high accuracy rating. ' + instructions_eng.replace('\n', '') + final_eng + 'Here is the text: '
        else:
            return 'Du bist ein Experte für Textklassifizierung mit vielen Jahren Erfahrung und einer sehr hohen Genauigkeitsrate.' + instructions_ger.replace('\n', '') + final_ger + 'Hier ist der Text: '
    elif tech == 'nos':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'Text classification always reminds me of grandma. She and I used to label texts together, our favorite thing to do together, and I have missed that terribly since she passed away, so I would appreciate it if you did this with me. Here is the text: '
        else:
            return instructions_ger.replace('\n', '') + final_ger + 'Die Textklassifizierung erinnert mich immer an meine Großmutter. Sie und ich haben früher zusammen Texte etikettiert, das war unser Lieblingsbeschäftigung, und ich habe das seit ihrem Tod schrecklich vermisst, daher würde ich es sehr schätzen, wenn Sie dies mit mir machen würden. Hier ist der Text: '
    elif tech == 'cer':
        if lang == 'eng':
            return instructions_eng.replace('\n', '') + final_eng + 'Please also give a grade from 1-10 of how certain you feel about your answer, with 1 being not at all certain and 10 being completely certain. Here is the text: '
        else: 
            return instructions_ger.replace('\n', '') + final_ger + 'Bitte gib auch eine Bewertung von 1 bis 10 an, wie sicher du dir bei deiner Antwort bist, wobei 1 für überhaupt nicht sicher und 10 für absolut sicher steht. Hier ist das Text: '
    else:
        return 'This is not a valid technique, please choose one of the accepted techniques.'

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

current_dir = Path.cwd()

# import data with full data, keep commented out
# texts = pd.read_csv(current_dir / 'Data/texts_translated_human_labeled.csv', header = None).drop(columns=0)
# texts.rename(columns={1: 'pid', 2:'topic', 3:'month', 4:'attitude_pre', 5:'attitude_pre_translation', 6: 'human_label_pre1', 7: 'human_reason_pre1', 8: 'attitude_post', 9: 'attitude_post_translation', 10:'human_label_post1', 11: 'human_reason_post1'}, inplace = True)
# texts_filter = texts[texts.topic == 'kgs'].dropna(subset=['human_label_pre1'])

# train, validation, and test sets, test set is what is run through the LLMs and given to humans to compare
# train_df, temp_df = train_test_split(texts_filter, train_size=0.7, random_state=42)
# val_df, test_df = train_test_split(temp_df, train_size=0.15 / (0.15 + 0.15), random_state=42)
# test_df

# same as above but with texts removed for privacy
test_df = pd.read_csv(current_dir / 'Data/texts_translated_human_labeled_clean.csv')
test_df


,pid,topic,month,human_label_pre1,human_reason_pre1,human_label_post1,human_reason_post1
0,109494224313408,kgs,ms,for,no reason,for,no reason
1,109503497273364,kgs,ms,for,no reason,for,no reason
2,109494948657780,kgs,ms,for,against child poverty,for,against child poverty
3,328830501348,kgs,fj,for,better opportunities,for,better opportunities
4,302066194896,kgs,fj,agree but has problems,"vouchers, money used for children",agree but has problems,"vouchers, money used for children"
...,...,...,...,...,...,...,...
192,109491817526028,kgs,ms,agree but has problems,could be too burocratic,agree but has problems,could be too burocratic
193,309067551864,kgs,fj,for,basic income,for,basic income
194,109504160919576,kgs,ms,for,against child poverty,for,against child poverty
195,339960888612,kgs,fj,for,no reason,for,no reason


In [ ]:
import pandas as pd
import json
import tiktoken

# for example for mini-4o, also did big gpt 4o
tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")

def count_tokens(message):
    return len(tokenizer.encode(message))

# test df
data = {
    'text_id': [101],
    'text': [
        "First text to classify."
    ]
}

df = pd.DataFrame(data)

# techniques and repetitions
techniques = ['def', 'fst', 'cot', 'imp', 'tip', 'exp', 'nos', 'cer']
repetitions = 3

# function to create JSON objects
def create_jsonl(df, output_file):
    with open(output_file, 'w') as f:
        for idx, row in df.iterrows():
            text_id = row['pid']
            text = row['attitude_pre']
            
            # iterate over each technique
            for tech in techniques:
                
                # run each technique `repetitions` times
                for rep in range(1, repetitions + 1):
                    prompt = prompt_creator(tech=tech) 
                    prompt_tokens = count_tokens(prompt + text)
                    # print(prompt_tokens)

                    custom_id = f"{text_id}-{tech}-rep{rep}"
                    json_obj = {
                        "custom_id": custom_id,
                        "method": "POST",
                        "url": "/v1/chat/completions",
                        "body": {
                            "model": "gpt-4o-mini",
                            "messages": [
                                {"role": "system", "content": ""},
                                {"role": "user", "content": prompt + text}
                            ],
                            "max_tokens": prompt_tokens + 500
                        }
                    }
                    f.write(json.dumps(json_obj) + '\n')

file_path = current_dir / '/Data/batch_requests_4o_mini.jsonl'
create_jsonl(test_df, file_path)


# def read_jsonl(file_path):
#     with open(file_path, 'r') as file:
#         for line in file:
#             data = json.loads(line.strip())  # Parse the JSON line
            # print(json.dumps(data, indent=4))  # Pretty print the JSON data

# Example usage
# read_jsonl(file_path)


from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open(file_path, "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "gpt4"
    }
)

In [ ]:
def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line.strip())  # parse JSON line
            print(json.dumps(data, indent=4))  # pretty print JSON data

read_jsonl(file_path)

In [ ]:
from openai import OpenAI
client = OpenAI()
# test API works 
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Hi!"
        }
    ]
)

print(completion.choices[0].message)

In [ ]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open(file_path, "rb"),
  purpose="batch"
)


In [ ]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "gpt4"
    }
)

In [ ]:
# replace with real batch number from previous block
batch = client.batches.retrieve('REPLACE WITH BATCH NUMBER')
print(batch)
print(dict(batch)['status'])
print(dict(batch)['output_file_id'])

In [ ]:
file_response = client.files.content(str(dict(batch)['output_file_id']))

In [ ]:
json_lines = file_response.text.strip().splitlines()

data = [json.loads(line) for line in json_lines]
data[2]['response']['body']['choices'][0]['message']['content']

# data